In [1]:
import numpy as np
import pandas

In [2]:
#Matrices
def Rx(q):
    mtx = np.identity(4)
    mtx[1, 1] = mtx[2, 2] = np.cos(q)
    mtx[2, 1],  mtx[1, 2] = np.sin(q), -np.sin(q)
    return mtx

def Rz(q):
    mtx = np.identity(4)
    mtx[0, 0] = mtx[1, 1] = np.cos(q)
    mtx[1, 0],  mtx[0, 1] = np.sin(q), -np.sin(q)
    return mtx

def Tz(z):
    mtx = np.identity(4)
    mtx[2,3] = z
    return mtx
def Ttool(*args):
    x,y,z = args
    mtx = np.identity(4)
    mtx[0,3],mtx[1,3],mtx[2,3] = x,y,z
    return mtx

def vectors(mtx):
    return mtx[:3,3], mtx[:3,:3]

In [3]:
def ForwardKinematics(*args): #from previous manipulator
    q1,q2,q3,q4,q5,q6,L1,L2,L3,d = args
    Tbase = np.identity(4)
    return Tbase @ Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2) @ Rx(q3) @ Tz(L3) @ Rz(q4) @ Rx(q5) @ Rz(q6) @ Ttool(*d)

In [4]:
#Solution FK
q1,q2,q3,q4,q5,q6,L1,L2,L3,d = [0.1,np.pi/7,1.57,0,-np.pi/4,np.pi/3,1,1,1,(0,0,1)]
FK = ForwardKinematics(*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])
FK

array([[ 0.46888175, -0.87822284,  0.09420481,  0.22750217],
       [ 0.3351654 ,  0.07823014, -0.93890585, -2.26743326],
       [ 0.81719891,  0.47181001,  0.3310306 ,  1.79883333],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [5]:
def Jacobian(FK, *args):

    q1,q2,q3,q4,q5,q6,L1,L2,L3,d = args

    On, rotation_mtx = vectors(FK)

    O1, rot1 = vectors(Rz(q1) @ Tz(L1))
    U1 = rot1[:, 2]
    J1 = np.concatenate((np.cross(U1, On-O1),
                         U1))

    O2, rot2 = vectors(Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2))
    U2 = rot2[:, 0]
    J2 = np.concatenate((np.cross(U2, On-O2),
                         U2))

    O3, rot3 = vectors(Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2) @ Rx(q3) @ Tz(L3))
    U3 = rot3[:, 0]
    J3 = np.concatenate((np.cross(U3, On-O3),
                         U3))

    O4, rot4 = vectors(Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2) @ Rx(q3) @ Tz(L3) @ Rz(q4))
    U4 = rot4[:, 2]
    J4 = np.concatenate((np.cross(U4, On-O4),
                         U4))

    O5, rot5 = vectors(Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2) @ Rx(q3) @ Tz(L3) @ Rz(q4) @ Rx(q5))
    U5 = rot5[:, 2]
    J5 = np.concatenate((np.cross(U5, On-O5),
                         U5))

    O6, rot6 = vectors(Rz(q1) @ Tz(L1) @ Rx(q2) @ Tz(L2) @ Rx(q3) @ Tz(L3) @ Rz(q4) @ Rx(q5) @ Rz(q6))
    U6 = rot6[:, 2]
    J6 = np.concatenate((np.cross(U6, On-O6),
                         U6))

    return pandas.DataFrame({0:J1,1:J2,2:J3,3:J4,4:J5,5:J6})

In [9]:
q1,q2,q3,q4,q5,q6,L1,L2,L3,d

On, rotation_mtx = vectors(FK)

O1, rot1 = vectors(Rz(q1) @ Tz(L1))
U1 = rot1[:, 2]
J1 = np.concatenate((np.cross(U1, On-O1),
                     U1))
J1

array([ 0.58957394,  0.06917146, -0.        ,  0.        ,  0.        ,
        1.        ])

In [6]:
Jacobian(FK,*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])

,0,1,2,3,4,5
0,2.267433,-0.010197,0.033048,-7.035742e-01,-1.110223e-16,-1.110223e-16
1,0.227502,0.101625,-0.329377,-7.059289e-02,0.000000e+00,0.000000e+00
2,-0.000000,-1.844934,-0.943620,2.775558e-17,4.163336e-17,4.163336e-17
3,0.000000,0.995004,0.995004,8.998127e-02,9.420481e-02,9.420481e-02
4,0.000000,0.099833,0.099833,-8.968113e-01,-9.389059e-01,-9.389059e-01
5,1.000000,0.000000,0.000000,-4.331661e-01,3.310306e-01,3.310306e-01


In [7]:
#Let's rotate one of the axis at pi/2
q1,q2,q3,q4,q5,q6,L1,L2,L3,d = [0.1,np.pi/7,1.57,0,np.pi/2,0,1,1,1,(0,0,1)]
FK = ForwardKinematics(*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])
J = Jacobian(FK,*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])
det = np.linalg.det(np.array(J, dtype = np.float32))
if det: # If det = 0
    print('There is no singularity, det = {}'.format(det))
else:
    print('Singularity detected, det = {}'.format(det))

Singularity detected, det = 0.0


In [8]:
q1,q2,q3,q4,q5,q6,L1,L2,L3,d = [0.1,0.1,0.1,0.1,0.1,0.1,1,1,1,(0,0,1)]
FK = ForwardKinematics(*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])
J = Jacobian(FK,*[q1,q2,q3,q4,q5,q6,L1,L2,L3,d])
det = np.linalg.det(np.array(J, dtype = np.float32))
if det:
    print('There is no singularity, det = {}'.format(det))
else:
    print('Singularity detected, det = {}'.format(det))

There is no singularity, det = 1.2603033672783034e-20
